Building a Question-Answering System with LlamaIndex


1.Import necessary classes and functions from the llama_index and langchain ibraries.

In [ ]:
!pip install --upgrade llama-index #Gérer l'indexation et la récupération des documents
!pip install llama-index-llms-huggingface #Charger et exécuter un modèle d'IA local
!pip install llama-index-llms-huggingface-api
!pip install llama-index transformers accelerate bitsandbytes sentencepiece # Optimiser les performances pour exécuter des modèles sur GPU
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

In [ ]:
pip install llama-index

In [ ]:
pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, StorageContext, load_index_from_storage, Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
from llama_index.core import download_loader

In [ ]:
import sys
import os
import shutil

Set up the HuggingFace API key

In [ ]:
os.environ["HUGGING_FACE_HUB_TOKEN"] = "xxxxxxxxxxxxxxxx"

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving 2307.14334.pdf to 2307.14334.pdf


In [ ]:
# Creating the folder 'papers' if not already present
os.makedirs("papers", exist_ok=True)

In [ ]:
# Moving the uploaded PDF into the 'papers' directory
shutil.move("2307.14334.pdf", "papers/2307.14334.pdf")

# Now our PDF is inside a folder called papers(SimpleDirectoryReader expects.)

'papers/2307.14334.pdf'

In [ ]:
# Loading the Document Using SimpleDirectoryReader
print(" Starting the document loading process...")

# Reading the document from the 'papers' directory
documents = SimpleDirectoryReader('papers').load_data()

# Confirming the number of documents loaded
len(documents)

 Starting the document loading process...


37

In [ ]:
# Choosing HuggingFace model (open-access)
model_name = "HuggingFaceH4/zephyr-7b-beta"

In [ ]:
# Initializing the LLMPredictor using HuggingFaceLLM

# Initializing the LLM configuration
llm_predictor = HuggingFaceLLM(
    model_name=model_name,
    tokenizer_name=model_name,
    context_window=3900,                   # how much text the model can consider
    max_new_tokens=256,                    # how much text the model will generate
    generate_kwargs={"temperature": 0.3},  # how creative the responses should be
    device_map="auto"                      # letting it automatically use GPU/CPU
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# 5- Creating a ServiceContext to bundle the LLM setup ((Modern Way: Configuring Settings with LLM predictor))
# Setting the LLM Predictor (Zephyr)
from llama_index.core import Settings
Settings.llm = llm_predictor

In [ ]:
# 6 - Indexing the loaded documents using the configured LLM and embeddings

from llama_index.core import VectorStoreIndex

print("Starting the indexing process...")

Starting the indexing process...


# 6 - Fixing embedding model to avoid OpenAI API key error

In [ ]:
!pip install sentence-transformers

In [ ]:
# Import required modules
from sentence_transformers import SentenceTransformer
from llama_index.core.base.embeddings.base import BaseEmbedding
from typing import List
import numpy as np
import asyncio

In [ ]:
# Updated version with all required methods
class MyHuggingFaceEmbedding(BaseEmbedding):
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)

    def _get_text_embedding(self, text: str) -> List[float]:
        return self.model.encode(text).tolist()

    def _get_query_embedding(self, query: str) -> List[float]:
        return self.model.encode(query).tolist()

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts).tolist()

    async def _aget_query_embedding(self, query: str) -> List[float]:
        # Simple sync fallback for async requirement
        return self._get_query_embedding(query)


In [ ]:
from llama_index.core import Settings

# Apply our custom embedding model that meets all LlamaIndex requirements
Settings.embed_model = MyHuggingFaceEmbedding()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ValueError: "MyHuggingFaceEmbedding" object has no field "model"